In [1]:
from py2neo import Graph
graph = Graph("http://localhost:7474", auth=("neo4j", "990910lhylhy"))
import numpy as np
import time

In [2]:
kw = 'China'
c1 = 'MATCH (a)-[r]->(n) WHERE a.entityName=\'' + kw + '\' RETURN a,r,n,labels(n),n.entityImportance'
start = time.time()
s1 = graph.run(c1).data()
mid = time.time()
print(mid-start)
# 第一次run需要一分钟左右

15.99192214012146


In [15]:
s1

[{'a': (_701045:Place {entityId: 802462, entityImportance: '18.369384217320146', entityName: 'China', entityProperty: 'longName: Peoples Republic of China,  name: the Peoples Republic of China,  demonym: Chinese,  leaderTitle: President,  leaderTitle: Premier,  leaderTitle: Congress Chairman,  leaderTitle: Conference Chairman,  leaderTitle: First-ranked Secretary of theSecretariat,  foundingDate: 1912-01-01,  areaTotal: 9.596961E12,  areaTotal: 9.596960073955787E12,  percentageOfAreaWater: 2.8,  flag: Flag of the People%27s Republic of China.svg,  populationTotal: 1376049000,  populationDensity: 2013.0,  populationDensity: 144.01610513633233', entityType: 'Country'}),
  'r': (_701045)-[:birthPlace {inverse: 'directed'}]->(_3115383),
  'n': (_3115383:Person {entityId: 3937616, entityImportance: '5.705273529344671', entityName: 'Zhang Jun (politician)', entityProperty: 'name: Zhang Jun,  office: Deputy Secretary of theCentral Commission for Discipline Inspection', entityType: 'OfficeHold

In [33]:
n = []
r = []
importance = []
label = []
a = []
for i in range(len(s1)):
    a.append(s1[i]['a'])
    n.append(s1[i]['n'])
    r.append(s1[i]['r'])
    importance.append(float(s1[i]['n.entityImportance']))
    label.append(s1[i]['labels(n)'][0])

In [34]:
entity_temp = n
relationship_temp = r
coverage = [1 for i in range(len(importance))]
selected_entity = list()
selected_relationship = list()
# if number of entities is bigger than limit number of entities
if len(importance)>=50:
    k = 0
    lamb = 0.6
    while k<50:
        values = list()
        # calculate value of all entity and select the biggest one
        for i in range(len(importance)):
            v = lamb*importance[i]+(1-lamb)*coverage[i]
            values.append(v)
        max_index = values.index(max(values))
        # add selected entity into list 'selected'
        selected_entity.append(entity_temp[max_index])
        selected_relationship.append(relationship_temp[max_index])
        # store label name as a variable
        label_name = label[max_index]
        # delete selected entity from four lists
        del entity_temp[max_index]
        del relationship_temp[max_index]
        del importance[max_index]
        del label[max_index]
        del coverage[max_index]
        # change coverage of entity with selected label to 0
        for i in range(len(coverage)):
            if label[i]==label_name:
                coverage[i] = 0
        k += 1
else:
    selected_entity = entity_temp
    selected_relationship = relationship_temp
selected_entity

[(_573488:Organisation {entityId: 626947, entityImportance: '10704.166594729104', entityName: 'Silver Ash', entityProperty: 'name: Silver Ash,  name: Yin Se Hui Chen ,  name: Yinse huichen,  activeYearsStartYear: 2000', entityType: 'Band'}),
 (_4005027:Organisation {entityId: 4934497, entityImportance: '7369.9985723785185', entityName: 'Cookies (Hong Kong band)', entityProperty: 'name: Cookies or cooklets,  alias: Mini Cookies (2003-2004),  activeYearsStartYear: 2002,  activeYearsEndYear: 2002', entityType: 'Band'}),
 (_2209784:Person {entityId: 2755704, entityImportance: '4949.576876741886', entityName: 'Ai Qing', entityProperty: 'name: Ai Qing,  pseudonym: Ejia (E Jia ),  pseudonym: Kea (Ke A ),  pseudonym: Linbi (Lin Bi ),  birthName: Jiang Zhenghan (Jiang Zheng Han ),  birthDate: 1910-03-27,  deathDate: 1996-05-05,  activeYearsStartYear: 1936,  activeYearsEndYear: 1986', entityType: 'Writer'}),
 (_1073920:Person {entityId: 1285757, entityImportance: '4559.282771088714', entityName:

In [51]:
selected = []
for i in range(len(selected_entity)):
    dic = {}
    dic['a'] = a[i]
    dic['r'] = selected_relationship[i]
    dic['n'] = selected_entity[i]
    
    selected.append(dic)

In [52]:
def buildSelf(nodeRecord):
    data = {"id": nodeRecord['a'].identity, "label": list(nodeRecord['a'].labels)[0],
            "name": nodeRecord['a']['entityName'], "entityId": nodeRecord['a']['entityId'],
            "entityImportance": nodeRecord['a']['entityImportance'],
            "entityProperty": nodeRecord['a']['entityProperty']}
    return {"data": data}

In [53]:
def buildNodes(nodeRecord):
    data = {"id": nodeRecord['n'].identity, "label": list(nodeRecord['n'].labels)[0],
            "name": nodeRecord['n']['entityName'], "entityId": nodeRecord['n']['entityId'],
            "entityImportance": nodeRecord['n']['entityImportance'],
            "entityProperty": nodeRecord['n']['entityProperty']}
    return {"data": data}


def buildEdges(relationRecord):
    data = {"id": relationRecord['r'].identity,
            "source": relationRecord['r'].start_node.identity,
            "target": relationRecord['r'].end_node.identity,
            "relationship": list(relationRecord['r'].types())[0],
            "direction": relationRecord['r']['inverse']}
    return {"data": data}

In [24]:
nodes = list(map(buildSelf, s1)) + list(map(buildNodes, selected))
edges = list(map(buildEdges, selected))

In [57]:
list(map(buildEdges,selected))

[{'data': {'id': 15293052,
   'source': 701045,
   'target': 573488,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 15471309,
   'source': 701045,
   'target': 4005027,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 15137408,
   'source': 701045,
   'target': 2209784,
   'relationship': 'deathPlace',
   'direction': 'directed'}},
 {'data': {'id': 17942731,
   'source': 701045,
   'target': 1073920,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 16668126,
   'source': 701045,
   'target': 3239685,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 22285390,
   'source': 701045,
   'target': 2722926,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 15375944,
   'source': 701045,
   'target': 2557450,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 21906183,
   'source': 701045,
   'target': 4181993,
   'relationship':

In [80]:
def select_node_edges(s1, k):
    # extract all data in lists
    a = []
    n = []
    r = []
    importance = []
    label = []
    for i in range(len(s1)):
        a.append(s1[i]['a'])
        n.append(s1[i]['n'])
        r.append(s1[i]['r'])
        importance.append(float(s1[i]['n.entityImportance']))
        label.append(s1[i]['labels(n)'][0])
    # select nodes and edges
    entity_temp = n
    relationship_temp = r
    coverage = [1 for i in range(len(importance))]
    selected_entity = list()
    selected_relationship = list()
    # if number of entities is bigger than limit number of entities
    if len(importance) >= k:
        count = 0
        lamb = 0.6
        while count < k:
            values = list()
            # calculate value of all entity and select the biggest one
            for i in range(len(importance)):
                v = lamb * importance[i] + (1 - lamb) * coverage[i]
                values.append(v)
            max_index = values.index(max(values))
            # add selected entity into list 'selected'
            selected_entity.append(entity_temp[max_index])
            selected_relationship.append(relationship_temp[max_index])
            # store label name as a variable
            label_name = label[max_index]
            # delete selected entity from four lists
            del entity_temp[max_index]
            del relationship_temp[max_index]
            del importance[max_index]
            del label[max_index]
            del coverage[max_index]
            # change coverage of entity with selected label to 0
            for i in range(len(coverage)):
                if label[i] == label_name:
                    coverage[i] = 0
            count += 1
    else:
        selected_entity = entity_temp
        selected_relationship = relationship_temp
    # integral as a new graph list
    selected = []
    for i in range(len(selected_entity)):
        dic = dict()
        dic['a'] = a[i]
        dic['n'] = selected_entity[i]
        dic['r'] = selected_relationship[i]
        selected.append(dic)
    return selected

In [83]:
sed = select_node_edges(s1, 50)

In [84]:
list(map(buildEdges,sed))

[{'data': {'id': 15293052,
   'source': 701045,
   'target': 573488,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 15471309,
   'source': 701045,
   'target': 4005027,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 15137408,
   'source': 701045,
   'target': 2209784,
   'relationship': 'deathPlace',
   'direction': 'directed'}},
 {'data': {'id': 17942731,
   'source': 701045,
   'target': 1073920,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 16668126,
   'source': 701045,
   'target': 3239685,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 22285390,
   'source': 701045,
   'target': 2722926,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 15375944,
   'source': 701045,
   'target': 2557450,
   'relationship': 'country',
   'direction': 'directed'}},
 {'data': {'id': 21906183,
   'source': 701045,
   'target': 4181993,
   'relationship':

In [85]:
list(map(buildSelf,sed))

[{'data': {'id': 701045,
   'label': 'Place',
   'name': 'China',
   'entityId': 802462,
   'entityImportance': '18.369384217320146',
   'entityProperty': 'longName: Peoples Republic of China,  name: the Peoples Republic of China,  demonym: Chinese,  leaderTitle: President,  leaderTitle: Premier,  leaderTitle: Congress Chairman,  leaderTitle: Conference Chairman,  leaderTitle: First-ranked Secretary of theSecretariat,  foundingDate: 1912-01-01,  areaTotal: 9.596961E12,  areaTotal: 9.596960073955787E12,  percentageOfAreaWater: 2.8,  flag: Flag of the People%27s Republic of China.svg,  populationTotal: 1376049000,  populationDensity: 2013.0,  populationDensity: 144.01610513633233'}},
 {'data': {'id': 701045,
   'label': 'Place',
   'name': 'China',
   'entityId': 802462,
   'entityImportance': '18.369384217320146',
   'entityProperty': 'longName: Peoples Republic of China,  name: the Peoples Republic of China,  demonym: Chinese,  leaderTitle: President,  leaderTitle: Premier,  leaderTitl

In [86]:
list(map(buildNodes,sed))

[{'data': {'id': 573488,
   'label': 'Organisation',
   'name': 'Silver Ash',
   'entityId': 626947,
   'entityImportance': '10704.166594729104',
   'entityProperty': 'name: Silver Ash,  name: Yin Se Hui Chen ,  name: Yinse huichen,  activeYearsStartYear: 2000'}},
 {'data': {'id': 4005027,
   'label': 'Organisation',
   'name': 'Cookies (Hong Kong band)',
   'entityId': 4934497,
   'entityImportance': '7369.9985723785185',
   'entityProperty': 'name: Cookies or cooklets,  alias: Mini Cookies (2003-2004),  activeYearsStartYear: 2002,  activeYearsEndYear: 2002'}},
 {'data': {'id': 2209784,
   'label': 'Person',
   'name': 'Ai Qing',
   'entityId': 2755704,
   'entityImportance': '4949.576876741886',
   'entityProperty': 'name: Ai Qing,  pseudonym: Ejia (E Jia ),  pseudonym: Kea (Ke A ),  pseudonym: Linbi (Lin Bi ),  birthName: Jiang Zhenghan (Jiang Zheng Han ),  birthDate: 1910-03-27,  deathDate: 1996-05-05,  activeYearsStartYear: 1936,  activeYearsEndYear: 1986'}},
 {'data': {'id': 1073